In [9]:
import spacy
import os
from spacy.tokens import DocBin
import json
import pandas as pd
import sys

In [4]:
json_filepath = os.path.join('combined.json')
json_file = open(json_filepath)

training_data = json.load(json_file)

In [7]:
def run_ner(training_data,
            model_name="en_core_web_sm", 
            train_spacy_filepath="./train.spacy", 
            config_filepath="./config.cfg", # ensure the config file has the correct model name
            output_filepath="./output"):
    
    !{sys.executable} -m spacy download {model_name}
    
    nlp = spacy.load(model_name)
    db = DocBin()

    for text, annot in training_data["annotations"]:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)

    db.to_disk(train_spacy_filepath)

    !{sys.executable} -m spacy train {config_filepath} --output {output_filepath} --paths.train {train_spacy_filepath} --paths.dev {train_spacy_filepath}

In [ ]:
run_ner(training_data)